# Open-Loop Control

## Preliminaries

The next lines setup some things and import the various libraries required to run this notebook.

In [ ]:
import sys
sys.path.append('..')

In [ ]:
%matplotlib inline  
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [ ]:
from ctrl.block.linear import Gain, Feedback, ShortCircuit, Differentiator
from ctrl.block.logger import Logger

Modify the variables `HOST` and `PORT` to reflect the network address of your device:

In [ ]:
from ctrl.client import Controller
#HOST, PORT = "localhost", 9999
HOST, PORT = "192.168.10.51", 9999
device = Controller(host = HOST, port = PORT)

## Static Response of the DC motor

It is helpful to think of the motor as an entity that takes an **input signal**, in this case signal `motor1` which is translated into a **voltage**, and produces an **output signal**, either an *angular position* or *angular velocity*. We represent such **input-output** relationship using a **block-diagram**:

<img src="figure1.png" width="300">

You will now perform some experiments to characterize the behavior of the DC motor. Instead of performing the experiment by hand, you will write python programs to vary the reference input, i.e. voltage, on the motor and then collect and analyze the resulting data.

The following code adds a signal `velocity1` and a logger:

In [ ]:
device.reset()
device.add_signal('velocity1')
device.add_filter('velocity1', Differentiator(), ['clock', 'encoder1'],['velocity1'])
device.add_sink('logger', Logger(auto_reset = True), ['clock', 'encoder1', 'velocity1', 'motor1'])

In [ ]:
print(device.info('all'))

**TASKS:**
1. Write python code that runs the motor at 100% for 2 seconds then at 95% for 2 second and so on until you reach 5%
<br/>
*Hint:* use a `for` loop and `range`
2. Import the data, use `matplotlib` to plot the `clock` *vs* `velocity1`
3. Plot also `motor1` *vs* `velocity1`
4. Compare and discuss your results with the other groups

**QUESTIONS:**
1. Is the response of the motor linear?
2. How could we calculate the *gain* of the motor?

### Linear Model

**TASKS:**
1. Use the following code to fit your data to a line of the form y = a x + b
2. Modify the code to fit a line of the form y = a x

**QUESTIONS:**
1. What does the code do?
2. Why is the value of b not zero?

In [ ]:
def func(x, a, b):
    return a * x + b

import scipy.optimize as optimize

ind = np.where(velocity1 > 0)
pars, residues = optimize.curve_fit(func, motor1[ind], velocity1[ind])
a, b = pars.tolist()
print('a = {:4.3f}, b = {:4.3f}'.format(a,b))

**TASKS:**
1. Use the values of a and b you calculated and the following code to plot the curve
   $$y = a \, x + b$$
   where $y$ is the velocity and $x$ the reference input along with the data you imported earlier
   
**QUESTIONS:**
1. How well do they match?
2. What is the meaning of the line when `motor1` > 0 but `velocity1` < 0
3. Repeat for a line of the form y = a x 

In [ ]:
plt.plot(motor1, velocity1, '.', np.array([0,100]), a * np.array([0,100]) + b,'-')
plt.xlabel('motor1 (%)')
plt.ylabel('velocity1 (Hz)')
plt.grid()

## Open-loop Control

In open-loop control the **reference signal**, in the case of the DC motor the input **voltage**, is produced by an algorithm, the **controller**, as shown in the following block-diagram:

<img src="figure2.png" width="500">

**TASKS:**
1. Use the line $$y = a x + b$$ where $y$ is the velocity and $x$ is the reference input to calculate the value of $x$ required to make the motor rotate at a constant 16Hz speed
2. Write python code that resets the controller logger, and sets the reference at the value you calculated for 5s
3. Import the data and plot time vs velocity and time vs reference
4. Discard the first second of data and use `np.mean` and `np.std` to calculate the average and standard deviation of the velocity

**QUESTIONS:**
1. How well did your calculation performed?
2. Generalize the calculation to work for any desired velocity: that's an **open-loop controller**

**TASKS:**
1. Run your code and have your group mate touch the top of the big pulley with his or her finger and apply a **gentle** force trying to stop the motor. Make sure the force **does not completely stop the motor**, as this may damage the motor
2. Repeat the above steps to calculate the mean and standard deviation of the velocity in this case


**QUESTIONS:**
1. Is the average close to 16HZ?
2. What happened to the velocity when your group mate touched the pulley?
3. What is causing the change in velocity?